In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import fsolve
from matplotlib.widgets import Button, Slider
from matplotlib.widgets import Slider
from super_cub import *

In [2]:
default_params = AircraftParams(
    bw=2.28,
    cw=0.25,
    a2dw=0.1,
    alpha_0w=-4,
    ew=0.9,
    cd0w=0.007,
    rw=0.002,
    Cmw=-0.1,
    bt=0.5,
    ct=0.1,
    cef=0.4*0.1,
    lt=2.5,
    tt=0.00635,
    ih=0.,
    et=0.99,
    a2dt=0.1,
    alpha_0t=-4,
    epsilon0=0,
    tau_e=0.7,
    cd0t=0.007,
    rt=0.002,
    df=0.2,
    lf=2,
    htv=0.2,
    ltv=0.8
)



In [3]:
airplane = super_cub(params)

In [6]:
def generate_data():
    alphas = np.linspace(-6.27, 15, 100)
    del_e = np.linspace(-10, 10, 5)
    Re_c = 500000
    airplane = super_cub(params)
    h = 0.4
    Cms = np.zeros((len(del_e), len(alphas)))
    Cls = np.zeros((len(del_e), len(alphas)))
    Cds = np.zeros((len(del_e), len(alphas)))

    for i in range(len(del_e)):
        for j in range(len(alphas)):
            Cms[i,j] = airplane.get_coefficients(alphas[j], del_e[i], Re_c, h)[0]
            Cls[i,j] = airplane.get_coefficients(alphas[j], del_e[i], Re_c, h)[1]
            Cds[i,j] = airplane.get_coefficients(alphas[j], del_e[i], Re_c, h)[2]
        


    # Define the del_e values
    del_e_values = np.linspace(-10, 10, 5)
    alpha_trim = np.linspace(-18,18, 100)
    trim_cl = []
    trim_cd = []

    for alpha in alpha_trim:
        f_CM = lambda x: airplane.get_coefficients(5, x, Re_c, h)[0]
        del_e = fsolve(f_CM, 0)[0]
        trim_cl.append(airplane.get_coefficients(alpha, del_e, Re_c, h)[1])
        trim_cd.append(airplane.get_coefficients(alpha, del_e, Re_c, h)[2])

    from scipy.optimize import curve_fit

    def parabolic_function(x, cd0, K):
        return cd0 + K*x**2

    popt, _ = curve_fit(parabolic_function, trim_cl, trim_cd)
    coefficients = np.polyfit(trim_cl, trim_cd, 2)

    # Extract the coefficients
    cd0 = popt[0]
    K = popt[1]

 

    return Cls, Cds, Cms, trim_cd, trim_cl, cd0, K

In [7]:
Cls, Cds, Cms, trim_cd, trim_cl, cd0, K = generate_data()

In [ ]:
%matplotlib widget
"""
Sliders for design hyperparameters:
    - b_w: wing span
    - c_w: wing chord
    - b_t: tail span
    - c_t: tail chord
    - c_ef: elevator chord factor (c_e/c_t)
    -i_h: horizontal tail incidence angle
    -d_f: fuselage diameter
    -l_f: fuselage length
    -h_tv: vertical tail height
    -l_tv: vertical tail length
"""
Sliders =       ['b_w',      'c_w',      'b_t',      'c_t',     'c_ef',  'i_h',      'd_f',     'l_f',      'h_tv',  'l_tv']
Slider_lims =   [(0.1, 2.28), (0.05, 1), (0.01, 2), (0.01, 1), (0.1, 1), (-10, 10), (0.01, 1), (0.01, 4), (0.01, 2), (0.01, 2)]
Slider_inits = [airplane.bw, airplane.cw, airplane.bt, airplane.ct, airplane.ce/airplane.ct, airplane.ih, airplane.df, airplane.lf, airplane.htv, airplane.ltv]
# Create the figure and axes
fig, axs = plt.subplots(2, 3, figsize=(12, 8))

# Plot Cm vs alphas for each del_e value
for i, del_e in enumerate(del_e_values):
    cm_line, = axs[0,0].plot(alphas, Cms[i], label=f"$\delta_e$ = {del_e}")
    
# Set the plot title and labels
axs[0,0].set_title("$C_m$ vs $\\alpha$")
axs[0,0].set_xlabel("$\\alpha$")
axs[0,0].set_ylabel("$C_m$")

# Add a legend
axs[0,0].legend()
axs[0,0].grid(True)

# Plot Cl vs alphas for each del_e value
for i, del_e in enumerate(del_e_values):
    cl_line, = axs[0,1].plot(alphas, Cls[i], label=f"$\delta_e$ = {del_e}")
    
# Set the plot title and labels
axs[0,1].set_title("$C_l$ vs $\\alpha$")
axs[0,1].set_xlabel("$\\alpha$")
axs[0,1].set_ylabel("$C_l$")

# Add a legend
axs[0,1].legend()
axs[0,1].grid(True)

# Plot Cl vs Cd for each del_e value
for i, del_e in enumerate(del_e_values):
    cd_line, = axs[1,0].plot(Cds[i], Cls[i], label=f"$\delta_e$ = {del_e}")
    
# Set the plot title and labels
axs[1,0].set_title("$C_l$ vs $C_d$")
axs[1,0].set_xlabel("$C_d$")
axs[1,0].set_ylabel("$C_l$")

# Add a legend
axs[1,0].legend()
axs[1,0].grid(True)

trim_line, = axs[1,1].plot(trim_cd, trim_cl, label='Data')
axs[1,1].plot(cd0+K*np.array(trim_cl)**2, trim_cl, label='Fitted Curve')

# Annotate the fit equations
equation = f"$C_d = {cd0:.5f} + {K:.5f}C_l^2$"
axs[1,1].annotate(equation, xy=(0.5, 0.5), xycoords='axes fraction', fontsize=12)

# Set the plot title and labels
axs[1,1].set_title("Trimmed Drag Polar with Fitted Curve")
axs[1,1].set_xlabel("$C_d$")
axs[1,1].set_ylabel("$C_l$")
axs[0, 2].axis('off')
axs[1, 2].axis('off')
# Adjust the spacing between subplots
# plt.tight_layout()
for i, slider_name in enumerate(Sliders):
    slider_ax = fig.add_axes([0.7, 0.7-(i*0.04), 0.8, 0.03])
    slider = Slider(slider_ax, slider_name, Slider_lims[i][0], Slider_lims[i][1], valinit=Slider_inits[i])


amp_slider = Slider(
    ax=axamp,
    label="Amplitude",
    valmin=0,
    valmax=10,
    valinit=init_amplitude,
    orientation="vertical"
)  
slider_ax = fig.add_axes([0.7, 0.7, 0.8, 0.03])
slider = Slider(slider_ax, 'Slider', 0, 1)
slider_ax2 = fig.add_axes([0.7, 0.7-0.04, 0.8, 0.03])
slider2 = Slider(slider_ax2, 'Slider2', 0, 1)
# Show the plot
plt.savefig('subplot_plots.jpg', dpi=300)



# Adjust the spacing between subplots
# plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
def update(val):
    params = default_params
    params.bw = bw_slider.val
    params.cw = cw_slider.val
    params.bt = bt_slider.val
    params.ct = ct_slider.val
    params.cef = cef_slider.val
    params.ih = ih_slider.val
    params.df = df_slider.val
    params.lf = lf_slider.val
    params.ltv = ltv_slider.val
    params.htv = htv_slider.val

    airplane = super_cub(params)
    Cls, Cds, Cms, trim_cd, trim_cl, cd0, K = generate_data()
    fig.canvas.draw_idle()
    